In [1]:
url = "https://www.google.com/search?q=wwdc+siri&tbs=cdr%3A1%2Ccd_min%3A6%2F10%2F2025%2Ccd_max%3A6%2F14%2F2025&tbm=nws"


In [51]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time
import re

driver_path = "/usr/local/bin/chromedriver" #output of "which chromedriver"
# 1. ChromeDriver Setup
service = Service(driver_path)  # Modifying the path to suit the local environment
options = Options()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 Chrome/120 Safari/537.36")

driver = webdriver.Chrome(service=service, options=options)

driver.get(url)
time.sleep(3)



In [39]:
next_button = driver.find_element(By.CSS_SELECTOR, "td[class='NKTSme'] a") #li.a-last a
next_button.click()

In [ ]:
# Find the news items (using CSS selectors, adjust as needed)
news_items = driver.find_elements(By.CSS_SELECTOR, "div.SoaBEf") #div.SoaBEf

page = 1
max_pages = 5  # Up to 5 Pages

# Extract data from each item
# while page <= max_pages:
news_data = []
for item in news_items:
    try:
        # Extract headline
        headline = item.find_element(By.CSS_SELECTOR, "div.n0jPhd.ynAwRc.MBeuO.nDgy9d").text #n0jPhd ynAwRc MBeuO nDgy9d

        # Extract source (adjust selector if needed)
        source = item.find_element(By.CSS_SELECTOR, "div.MgUUmf.NUnG9d").text

        # Extract snippet (adjust selector if needed)
        intro = item.find_element(By.CSS_SELECTOR, "div.GI74Re.nDgy9d").text

        news_data.append({
            "headline": headline,
            "source": source,
            "intro": intro
        })
    except Exception as e:
        print(f"Error extracting data: {e}")

    # # Clicking next page
    #     try:
    #         next_button = driver.find_element(By.CSS_SELECTOR, "td[class='NKTSme'] a") #all the pages are set with this class so it goes from page 1 -> 2 -> 1 -> .. 
    #         next_button.click()
    #         page += 1
    #         time.sleep(3)
    #     except NoSuchElementException:
    #         print("⛔️ No more pages ahead.")
    #         break

# 5. Converting the results into a DataFrame 
df = pd.DataFrame(news_data)

# 6. Saving into .csv
df.to_csv("siri_news.csv", index=False)

# Close the browser
# driver.quit()

In [53]:
df

,headline,source,intro
0,"WWDC: Yes but Siri will get smart one day, but...",Apple Must,At the edge of WWDC 2025 Apple confessed that ...
1,Apple targets spring 2026 for release of delay...,South China Morning Post,The company originally introduced the next-gen...
2,Apple Says Personalized Siri Features Shown at...,MacRumors,Apple first announced the personalized Siri fe...
3,iOS 26 screenshots could be an intriguing prev...,Engadget,"But, as someone who takes too many screenshots..."
4,Apple pushes Siri overhaul to 2026 after initi...,digitimes,Apple executives have confirmed that the compa...
5,WWDC 2025 Interview: Federighi and Joswiak on ...,iClarified,Apple's Craig Federighi and Greg Joswiak discu...
6,Apple unveils a new look but the wait for AI S...,The Sydney Morning Herald,"Cupertino, California: Apple has made a number..."
7,What WWDC Tells Us About the Future of Apple a...,Business Insider,The lack of a Siri upgrade at WWDC 2025 create...
8,Apple's WWDC faceplant costs it $75 billion,Fudzilla.com,Jobs' Mob shares nosedived six minutes into it...
9,Apple's hyped AI-powered Siri conspicuously ab...,Yahoo,Last year at its biggest software event of the...


In [ ]:
# 4. Collecting Reviews
news = []
page = 1
max_pages = 5  # Up to 5 Pages

while page <= max_pages:
    print(f"Collecting info on 📄 page {page}...")

    review_elements = driver.find_elements(By.CSS_SELECTOR, "li[data-hook='review']")
    for r in review_elements:
        try:
            # Title
            title = r.find_element(By.CSS_SELECTOR, "[data-hook='review-title']").text.strip()

            # Date
            date = r.find_element(By.CSS_SELECTOR, "[data-hook='review-date']").text.strip()

            # Body
            body = r.find_element(By.CSS_SELECTOR, "[data-hook='review-body']").text.strip()

            news.append({
                "title": title,
                "date": date,
                "body": body
            })

        except Exception as e:
            print("❌ Failed to parse reviews:", e)
            continue

    # Clicking next page
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "li.a-last a")
        next_button.click()
        page += 1
        time.sleep(3)
    except NoSuchElementException:
        print("⛔️ No more pages ahead.")
        break

# 5. Converting the results into a DataFrame 
df = pd.DataFrame(news)
print(df)

# 6. Saving into .csv
df.to_csv("amazon_reviews.csv", index=False)

# driver.quit()


In [4]:
news

['WWDC: Yes but Siri will get smart one day, but not today']

In [16]:
news

[{'title': 'WWDC: Yes but Siri will get smart one day, but not today',
  'body': "At the edge of WWDC 2025 Apple confessed that it doesn't now expect the long-awaited contextually-aware Siri to ship before next year."}]